In [184]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive/')

  # Install the latest version of torchtext library quietly without showing output

  !pip install transformers evaluate wandb datasets accelerate  -U -qq  ## NEW LINES ##

  basepath = '/content/drive/MyDrive/NLP_Emotions'




Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [185]:
# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix
from datetime import datetime

# For plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

### NEW ##########################
# imports from Huggingface ecosystem
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

# wandb library
import wandb

In [186]:
base_folder = Path(basepath)
data_folder = base_folder/'datasets'
model_folder = base_folder/'models'
custom_functions = base_folder/'custom-functions'

In [187]:
model_folder.mkdir(exist_ok=True, parents = True)

In [188]:
model_folder

PosixPath('/content/drive/MyDrive/NLP_Emotions/models')

In [189]:
import pandas as pd

In [190]:
train = pd.read_csv('/content/drive/MyDrive/NLP_Emotions/datasets/train.csv')

In [191]:
train.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2017-21441,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,2017-31535,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,2017-21068,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,2017-31436,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,2017-22195,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


In [192]:
from sklearn.model_selection import train_test_split

In [193]:
data = train_test_split(train, test_size=0.2, random_state=42)
train_cleaned = data[0]
valid_cleaned = data[1]
print(train_cleaned.shape)
print(valid_cleaned.shape)


(6179, 13)
(1545, 13)


In [194]:
train_cleaned.describe()

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
count,6179.000000,6179.000000,6179.000000,6179.000000,6179.000000,6179.000000,6179.000000,6179.000000,6179.000000,6179.000000,6179.000000
mean,0.370125,0.142418,0.376274,0.171225,0.375142,0.110212,0.300696,0.114582,0.293413,0.052759,0.049846
std,0.482877,0.349506,0.484489,0.376736,0.484199,0.313179,0.458598,0.318542,0.455363,0.223571,0.217645
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [195]:
train_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6179 entries, 4291 to 7270
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            6179 non-null   object
 1   Tweet         6179 non-null   object
 2   anger         6179 non-null   int64 
 3   anticipation  6179 non-null   int64 
 4   disgust       6179 non-null   int64 
 5   fear          6179 non-null   int64 
 6   joy           6179 non-null   int64 
 7   love          6179 non-null   int64 
 8   optimism      6179 non-null   int64 
 9   pessimism     6179 non-null   int64 
 10  sadness       6179 non-null   int64 
 11  surprise      6179 non-null   int64 
 12  trust         6179 non-null   int64 
dtypes: int64(11), object(2)
memory usage: 675.8+ KB


In [196]:
# X y Split
X_train = train_cleaned['Tweet']
X_valid = valid_cleaned['Tweet']

In [197]:
y_train = train_cleaned[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
                    'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].values.tolist()

y_valid = valid_cleaned[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
                    'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].values.tolist()


#Hugging Face

In [198]:
train = Dataset.from_dict({'texts': X_train, 'labels': y_train})
valid = Dataset.from_dict({'texts': X_valid, 'labels': y_valid})

In [199]:
train

Dataset({
    features: ['texts', 'labels'],
    num_rows: 6179
})

In [200]:
train[0]

{'texts': "Going to get myself a copy of @StephenKing's CUJO for an upcoming project that I can't talk about just yet.  #amwriting",
 'labels': [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0]}

In [201]:
train[1]

{'texts': "@carysmithwriter @Maria_Savva @RealRockAndRoll We're the least known band in the World, but so glad you asked #muchlove ",
 'labels': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]}

## <Font color = 'indianred'>**4. Create Custom Model and Model Config Class** </font>
embedding_layer-->linear--> ReLU -->dropout --> batch norm --> linear-->ReLu-->Dropout--> batchnorm ---> linear layer

The new `CustomMLP` model represents a significant adaptation to align with the conventions of the Hugging Face ecosystem, which differs from traditional PyTorch models in several key ways:

### High-Level Changes for Hugging Face Compatibility:

1. **Output of Both Loss and Logits**:
   - In the Hugging Face framework, models typically output both the loss and the logits, as opposed to traditional models which might only output logits.

2. **Requirement of a Configuration Class**:
   - Hugging Face models rely on a configuration class, like `CustomConfig`, to handle various model settings and hyperparameters. This configuration class is essential for defining model-specific parameters and ensures that the model can be easily adjusted, saved, and loaded with these configurations.

3. **Inheritance from `PreTrainedModel`**:
   - Models in the Hugging Face ecosystem are typically subclasses of `PreTrainedModel` rather than the standard `nn.Module` used in PyTorch. This subclassing provides additional functionalities crucial for Hugging Face models, such as compatibility with pre-trained weights, standardized methods for loading/saving models, and integration with the rest of the Hugging Face libraries.

In [202]:
from transformers import PretrainedConfig

class CustomConfig(PretrainedConfig):
    def __init__(self, vocab_size=0, embedding_dim=0, hidden_dim1=0, hidden_dim2=0, num_labels=11, **kwargs):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.num_labels = num_labels

In [203]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput

class CustomMLP(PreTrainedModel):
    config_class = CustomConfig

    def __init__(self, config):
        super().__init__(config)

        self.embedding_bag = nn.EmbeddingBag(config.vocab_size, config.embedding_dim)
        self.layers = nn.Sequential(
            nn.Linear(config.embedding_dim, config.hidden_dim1),
            nn.BatchNorm1d(num_features=config.hidden_dim1),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(config.hidden_dim1, config.hidden_dim2),
            nn.BatchNorm1d(num_features=config.hidden_dim2),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(config.hidden_dim2, 11)  # Output layer
        )
    def forward(self, input_ids, offsets, labels=None):
      embed_out = self.embedding_bag(input_ids, offsets)
      logits = self.layers(embed_out)
      loss = None
      if labels is not None:
        if labels.dim() == 1:
          labels = labels.unsqueeze(1)

        # Print shapes for debugging
        #print(f"logits shape: {logits.shape}, labels shape: {labels.shape}")

        # Ensure labels are float
        labels = labels.float()

        # Calculate the loss using BCEWithLogitsLoss
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels)  # logits already include sigmoid

      return SequenceClassifierOutput(
        loss=loss,
        logits=logits
    )



#Collate Function

In [204]:
from collections import Counter, OrderedDict
from typing import Dict, List, Optional, Union

class Vocab:
    def __init__(self, tokens: List[str]) -> None:
        self.itos: List[str] = tokens
        self.stoi: Dict[str, int] = {token: i for i, token in enumerate(tokens)}
        self.default_index: Optional[int] = None

    def __getitem__(self, token: str) -> int:
        if token in self.stoi:
            return self.stoi[token]
        if self.default_index is not None:
            return self.default_index
        raise RuntimeError(f"Token '{token}' not found in vocab")

    def __contains__(self, token: str) -> bool:
        return token in self.stoi

    def __len__(self) -> int:
        return len(self.itos)

    def insert_token(self, token: str, index: int) -> None:
        if index < 0 or index > len(self.itos):
            raise ValueError("Index out of range")
        if token in self.stoi:
            old_index = self.stoi[token]
            if old_index < index:
                self.itos.pop(old_index)
                self.itos.insert(index - 1, token)
            else:
                self.itos.pop(old_index)
                self.itos.insert(index, token)
        else:
            self.itos.insert(index, token)

        self.stoi = {token: i for i, token in enumerate(self.itos)}

    def append_token(self, token: str) -> None:
        if token in self.stoi:
            raise RuntimeError(f"Token '{token}' already exists in the vocab")
        self.insert_token(token, len(self.itos))

    def set_default_index(self, index: Optional[int]) -> None:
        self.default_index = index

    def get_default_index(self) -> Optional[int]:
        return self.default_index

    def lookup_token(self, index: int) -> str:
        if 0 <= index < len(self.itos):
            return self.itos[index]
        raise RuntimeError(f"Index {index} out of range")

    def lookup_tokens(self, indices: List[int]) -> List[str]:
        return [self.lookup_token(index) for index in indices]

    def lookup_indices(self, tokens: List[str]) -> List[int]:
        return [self[token] for token in tokens]

    def get_stoi(self) -> Dict[str, int]:
        return self.stoi.copy()

    def get_itos(self) -> List[str]:
        return self.itos.copy()

    @classmethod
    def vocab(cls, ordered_dict: Union[OrderedDict, Counter], min_freq: int = 1, specials: Optional[List[str]] = None, special_first: bool = True) -> 'Vocab':
        specials = specials or []
        for token in specials:
            ordered_dict.pop(token, None)

        tokens = [token for token, freq in ordered_dict.items() if freq >= min_freq]

        if special_first:
            tokens = specials + tokens
        else:
            tokens = tokens + specials

        return cls(tokens)

In [205]:
def get_vocab(dataset, min_freq=1):
    """
    Generate a vocabulary from a dataset.

    Args:
        dataset (Dataset): A Hugging Face Dataset object. The dataset should
                           have a key 'texts' that contains the text data.
        min_freq (int): The minimum frequency for a token to be included in
                        the vocabulary.

    Returns:
        torchtext.vocab.Vocab: Vocabulary object containing tokens from the
                               dataset that meet or exceed the specified
                               minimum frequency. It also includes a special
                               '<unk>' token for unknown words.
    """
    # Initialize a counter object to hold token frequencies
    counter = Counter()

    # Update the counter with tokens from each text in the dataset
    # Iterating through texts in the dataset
    for text in dataset['texts']:  ###### Change from previous function ####
        counter.update(str(text).split())

    # Create a vocabulary using the counter object
    # Tokens that appear fewer times than `min_freq` are excluded
    my_vocab = Vocab.vocab(counter, min_freq=min_freq)

    # Insert a '<unk>' token at index 0 to represent unknown words
    my_vocab.insert_token('<unk>', 0)

    # Set the default index to 0
    # This ensures that any unknown word will be mapped to '<unk>'
    my_vocab.set_default_index(0)

    return my_vocab

In [206]:
def tokenizer(text, vocab):
    """Converts a tweet to a list of indices using a vocabulary dictionary."""
    return [vocab[token] for token in str().split() if token in vocab]


In [207]:
def collate_batch(batch, my_vocab):
  labels = [sample['labels'] for sample in batch]
  texts = [sample['texts'] for sample in batch]

  labels = torch.tensor(labels, dtype =torch.float32) #check this later

  list_of_list_of_indices = [tokenizer(text, my_vocab) for text in texts]
 #check this later
    # Concatenate all text indices into a single tensor
  input_ids = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])

    # Compute the offsets for each text in the concatenated tensor
  offsets = [0] + [len(i) for i in list_of_list_of_indices]
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)

  return {
        'input_ids': input_ids,
        'offsets': offsets,
        'labels': labels
  }

In [208]:
emotion_vocab = get_vocab(train, min_freq=2)
collate_fn = partial(collate_batch, my_vocab=emotion_vocab)

In [209]:
my_config = CustomConfig(vocab_size=len(emotion_vocab),
                         embedding_dim=300,
                         hidden_dim1=200,
                         hidden_dim2=100,
                         num_labels=11)



In [210]:
my_config

CustomConfig {
  "embedding_dim": 300,
  "hidden_dim1": 200,
  "hidden_dim2": 100,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10"
  },
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "transformers_version": "4.45.2",
  "vocab_size": 8134
}

In [211]:
my_config.id2label = {
    0: 'anger',
    1: 'anticipation',
    2: 'disgust',
    3: 'fear',
    4: 'joy',
    5: 'love',
    6: 'optimism',
    7: 'pessimism',
    8: 'sadness',
    9: 'surprise',
    10: 'trust'
}


In [212]:
my_config.label2id = {v: k for k, v in my_config.id2label .items()}

In [213]:
my_config

CustomConfig {
  "embedding_dim": 300,
  "hidden_dim1": 200,
  "hidden_dim2": 100,
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "transformers_version": "4.45.2",
  "vocab_size": 8134
}

In [214]:
model = CustomMLP(config=my_config)


In [215]:
model

CustomMLP(
  (embedding_bag): EmbeddingBag(8134, 300, mode='mean')
  (layers): Sequential(
    (0): Linear(in_features=300, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=100, out_features=11, bias=True)
  )
)

In [216]:
from sklearn.metrics import accuracy_score, f1_score

# Define the multi-label compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    sigmoid_logits = torch.sigmoid(torch.tensor(logits))
    predictions = (sigmoid_logits >= 0.5).int().numpy()
    labels = labels.astype(int)

    # F1 score (macro and weighted are common for multi-label tasks)
    f1_micro = f1_score(labels, predictions, average='micro', zero_division=0)
    f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)
    # Combine metrics in a dictionary
    return {
        'f1_macro': f1_macro,
        'f1_micro': f1_micro
    }

# Example output
# logits would be the model's output and labels would be the ground truth from the evaluation set
# logits, labels = eval_pred
# print(compute_metrics((logits, labels)))


In [217]:
# Configure training parameters
training_args = TrainingArguments(
    num_train_epochs=100,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    weight_decay=0.5,
    learning_rate=0.0005,
    optim='adamw_torch',
    remove_unused_columns=False,

    output_dir=str(model_folder),
    evaluation_strategy='steps',
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    save_total_limit=2,

    metric_for_best_model="eval_f1_micro",
    greater_is_better=True,

    logging_strategy='steps',
    logging_steps=200,
    report_to='wandb',
    run_name='multi_hf_trainer',
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [218]:
trainer = Trainer(
    model=model,  # Ensure this is the correct model
    args=training_args,  # Ensure training arguments are properly set
    train_dataset=train,  # Dataset for training
    eval_dataset=valid,  # Dataset for evaluation
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)

In [219]:
trainer.train()

Step,Training Loss,Validation Loss,F1 Macro,F1 Micro
200,0.530300,1.402923,0.000000,0.000000
400,0.480700,1.040035,0.000000,0.000000
600,0.476900,0.614325,0.000000,0.000000
800,0.476600,0.796621,0.000000,0.000000
1000,0.476100,0.665494,0.000000,0.000000
1200,0.474700,0.555374,0.000000,0.000000
1400,0.474700,0.495800,0.000000,0.000000
1600,0.474300,0.479613,0.000000,0.000000
1800,0.474000,0.477686,0.000000,0.000000
2000,0.473200,0.474177,0.000000,0.000000


TrainOutput(global_step=4900, training_loss=0.4764146906015824, metrics={'train_runtime': 565.4422, 'train_samples_per_second': 1092.773, 'train_steps_per_second': 8.666, 'total_flos': 0.0, 'train_loss': 0.4764146906015824, 'epoch': 100.0})

In [220]:
trainer.evaluate()

{'eval_loss': 1.4029228687286377,
 'eval_f1_macro': 0.0,
 'eval_f1_micro': 0.0,
 'eval_runtime': 0.485,
 'eval_samples_per_second': 3185.616,
 'eval_steps_per_second': 26.805,
 'epoch': 100.0}

In [221]:
valid_output = trainer.predict(valid)

In [222]:
valid_output._fields

('predictions', 'label_ids', 'metrics')

In [223]:
import numpy as np
valid_probs = valid_output.predictions
valid_preds = (valid_probs > 0.5).astype(int)
valid_labels = np.array(valid_output.label_ids).astype(int)


In [224]:
valid_output.metrics

{'test_loss': 1.4029228687286377,
 'test_f1_macro': 0.0,
 'test_f1_micro': 0.0,
 'test_runtime': 0.2319,
 'test_samples_per_second': 6662.191,
 'test_steps_per_second': 56.057}

In [227]:
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 200.


In [228]:
wandb.finish()

eval/f1_macro,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/f1_micro,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,█▅▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
eval/runtime,▅▁▂▂▂▂▁▁▁▁▂▆▅▅█▅▇▃▄▃▃▆▂▂▇
eval/samples_per_second,▂▇▄▆▅▆█▇▇█▆▂▂▂▁▂▁▃▂▃▄▁▅▅▁
eval/steps_per_second,▂▇▄▆▅▆█▇▇█▆▂▂▂▁▂▁▃▂▃▄▁▅▅▁
test/f1_macro,▁
test/f1_micro,▁
test/loss,▁
test/runtime,▁
test/samples_per_second,▁


Test Set

In [229]:
test = pd.read_csv('/content/drive/MyDrive/NLP_Emotions/datasets/test.csv')

In [230]:
test.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
1,2018-03739,"Academy of Sciences, eschews the normally sobe...",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
2,2018-00385,I blew that opportunity -__- #mad,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
3,2018-03001,This time in 2 weeks I will be 30... 😥,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
4,2018-01988,#Deppression is real. Partners w/ #depressed p...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


In [232]:
X_test = test['Tweet'].values

In [233]:
device = 'cpu'

list_of_list_of_indices = [tokenizer(text, emotion_vocab) for text in X_test]


offsets = [0] + [len(i) for i in list_of_list_of_indices]
offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
indices = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])
indices
device = 'cpu'
model.to(device)


model.eval()


outputs = model(indices, offsets)



In [241]:
probs = torch.sigmoid(outputs.logits)
threshold = 0.5
predicted_labels = (probs >= threshold).int()
no_labels = predicted_labels.sum(1) == 0
predicted_labels[no_labels, torch.max(probs[no_labels], dim=1).indices] = 1

In [242]:
binary_preds = predicted_labels
predicted_label_names = []
for prediction in binary_preds:
    labels = [model.config.id2label[i] for i, label in enumerate(prediction) if label == 1]
    predicted_label_names.append(labels)

In [243]:
result= pd.DataFrame(predicted_labels.numpy(), columns=my_config.id2label.values())
result.insert(0, 'ID', test['ID'])
result


,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,0,0,1,0,0,0,0,0,0,0,0
1,2018-03739,0,0,1,0,0,0,0,0,0,0,0
2,2018-00385,0,0,1,0,0,0,0,0,0,0,0
3,2018-03001,0,0,1,0,0,0,0,0,0,0,0
4,2018-01988,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-03848,0,0,1,0,0,0,0,0,0,0,0
3255,2018-00416,0,0,1,0,0,0,0,0,0,0,0
3256,2018-03717,0,0,1,0,0,0,0,0,0,0,0
3257,2018-03504,0,0,1,0,0,0,0,0,0,0,0


In [244]:
result.to_csv('submission.csv', index=False)